### What are the steps?

1. read the data
2. remove the log2 transformation
3. replace the nan values with 0, using the nan_mask
3. use the metadata to assin the important metadata
    - Study ID (or the cohort ID?) becomes the batch id
    - Cohort Label becomes the celltype label
    - Age, Sex, 
    - create the pre-train sets (train, val, test, NA)
    - create the finetune sets (train, val, test, NA)

4. save the data as a h5ad file
5. use the data in `finetune_integration.py` to create the vocab and pretrain the data
6. set up a VM to run the pretrain
7. create an scgpt directory within mz_embed_engine
8. add the modules needed for scgpt to the requirements.txt
9. update the docker image    

In [1]:
import pandas as pd
import anndata as ad
import numpy as np

In [25]:
input_dir = '/Users/jonaheaton/Desktop/scGPT-main/data/metabolomics_apr24'

intensity_data_file = 'alignment_36_grid_search_index_1_combined_study.csv'
nan_loc_file = 'alignment_36_grid_search_index_1_nan_mask.csv'
metadata2_file = 'metadata_df.csv'

In [26]:
metadata2 = pd.read_csv(input_dir+'/'+metadata2_file,index_col=0)

In [88]:
metadata2

,cohort_id,Study ID,Cohort Label,Cohort ID Expanded,Study ID Expanded,Cohort ID
file_name,,,,,,
0271_Marios_PD1_Inhib2_HP-AF002687-9.mzML,541,RCC,adult_cancer,RCC3,ST001237,541
0717_Marios_PD1_Inhib2_HP-AF002790-8.mzML,541,RCC,adult_cancer,RCC3,ST001237,541
0925_Marios_PD1_Inhib2_HP-AE673799-7.mzML,541,RCC,adult_cancer,RCC3,ST001237,541
0200a_Marios_RCC_HP-RPREFA11.mzML,541,RCC,adult_cancer,RCC1,ST001236,541
0722_Marios_PD1_Inhib2_HP-AF016226-8.mzML,541,RCC,adult_cancer,RCC3,ST001237,541
...,...,...,...,...,...,...
LungNodule_HILIC_Pos_91.mzML,581,ST000388,adult_cancer,581,ST000388,581
LungNodule_HILIC_Pos_66.mzML,581,ST000388,adult_cancer,581,ST000388,581
LungNodule_HILIC_Pos_53.mzML,581,ST000388,adult_cancer,581,ST000388,581


In [27]:
metadata2.shape

(18257, 6)

In [28]:
metadata2

,cohort_id,Study ID,Cohort Label,Cohort ID Expanded,Study ID Expanded,Cohort ID
file_name,,,,,,
0271_Marios_PD1_Inhib2_HP-AF002687-9.mzML,541,RCC,adult_cancer,RCC3,ST001237,541
0717_Marios_PD1_Inhib2_HP-AF002790-8.mzML,541,RCC,adult_cancer,RCC3,ST001237,541
0925_Marios_PD1_Inhib2_HP-AE673799-7.mzML,541,RCC,adult_cancer,RCC3,ST001237,541
0200a_Marios_RCC_HP-RPREFA11.mzML,541,RCC,adult_cancer,RCC1,ST001236,541
0722_Marios_PD1_Inhib2_HP-AF016226-8.mzML,541,RCC,adult_cancer,RCC3,ST001237,541
...,...,...,...,...,...,...
LungNodule_HILIC_Pos_91.mzML,581,ST000388,adult_cancer,581,ST000388,581
LungNodule_HILIC_Pos_66.mzML,581,ST000388,adult_cancer,581,ST000388,581
LungNodule_HILIC_Pos_53.mzML,581,ST000388,adult_cancer,581,ST000388,581


In [4]:
intensity_data = pd.read_csv(input_dir+'/'+intensity_data_file,index_col=0)

In [6]:
intensity_data.min().min()

4.719383293556531

In [5]:
nanloc_data = pd.read_csv(input_dir+'/'+nan_loc_file,index_col=0)

In [10]:
nanloc_data.sum()

0689_Marios_PD1_Inhib2_HP-E9791866-7.mzML     689
0917_Marios_PD1_Inhib2_HP-AD424471-8.mzML     261
1083_Marios_PD1_Inhib2_HP-AD827416-6.mzML     388
0198_Marios_RCC_HP-001300090-1.mzML           823
0001_Marios_PD1_Inhib2_HP-S3473952-7.mzML     637
                                             ... 
LungNodule_HILIC_Pos_83.mzML                 2067
LungNodule_HILIC_Pos_116.mzML                2072
LungNodule_HILIC_Pos_11.mzML                 2041
LungNodule_HILIC_Pos_63.mzML                 2103
LungNodule_HILIC_Pos_8.mzML                  2041
Length: 18257, dtype: int64

In [9]:
nanloc_data.head()

,0689_Marios_PD1_Inhib2_HP-E9791866-7.mzML,0917_Marios_PD1_Inhib2_HP-AD424471-8.mzML,1083_Marios_PD1_Inhib2_HP-AD827416-6.mzML,0198_Marios_RCC_HP-001300090-1.mzML,0001_Marios_PD1_Inhib2_HP-S3473952-7.mzML,0575_Marios_PD1_Inhib2_HP-AE436649-7.mzML,0701_Marios_PD1_Inhib2_HP-EA715505-8.mzML,0697_Marios_PD1_Inhib2_HP-EA495437-7.mzML,0273_Marios_PD1_Inhib2_HP-AD498710-7.mzML,0154_Marios_RCC_HP-001100010-3.mzML,...,LungNodule_HILIC_Pos_37.mzML,LungNodule_HILIC_Pos_56.mzML,LungNodule_HILIC_Pos_53.mzML,LungNodule_HILIC_Pos_22.mzML,LungNodule_HILIC_Pos_76.mzML,LungNodule_HILIC_Pos_83.mzML,LungNodule_HILIC_Pos_116.mzML,LungNodule_HILIC_Pos_11.mzML,LungNodule_HILIC_Pos_63.mzML,LungNodule_HILIC_Pos_8.mzML
FT10031,True,False,False,False,True,False,True,True,False,False,...,True,True,False,False,False,False,False,True,False,False
FT10035,False,False,False,True,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
FT10037,True,False,False,False,True,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
FT10039,False,False,False,True,False,False,False,False,False,True,...,True,True,True,True,True,True,True,True,True,True
FT10040,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False


In [11]:
intensity_data_raw = 2**intensity_data
intensity_data_raw[nanloc_data] = 0
intensity_data_raw.T.to_csv(input_dir+'/raw_data.csv')

In [14]:
intensity_data_raw = intensity_data_raw.T

In [15]:
intensity_data_raw.head()

,FT10031,FT10035,FT10037,FT10039,FT10040,FT10041,FT10042,FT10043,FT10044,FT10045,...,FT9976,FT9978,FT9980,FT9984,FT9985,FT9988,FT9989,FT9995,FT9997,FT9999
0689_Marios_PD1_Inhib2_HP-E9791866-7.mzML,0.000000e+00,416011.895284,0.000000e+00,534466.654178,6.425904e+05,203102.714974,2.610437e+06,210839.496980,0.000000,1.301819e+06,...,307959.794264,158014.882248,0.000000,474281.391310,236509.974965,369683.096046,512439.261761,7.056764e+05,0.000000,1.183896e+06
0917_Marios_PD1_Inhib2_HP-AD424471-8.mzML,1.773312e+06,542087.945713,4.800561e+06,967257.425662,2.321142e+05,214611.916062,1.516372e+06,434788.467626,200827.466887,7.563777e+05,...,664111.175328,236487.305832,986076.055189,864669.775886,128261.018605,749280.373961,822651.093813,1.199410e+06,436126.272931,7.367980e+05
1083_Marios_PD1_Inhib2_HP-AD827416-6.mzML,1.033554e+06,500571.466300,2.757909e+06,705723.602554,9.866347e+05,404477.201675,7.858213e+05,239150.264397,0.000000,3.391990e+05,...,457774.331638,156851.893930,0.000000,0.000000,79795.454343,251441.922080,480521.919763,7.904514e+05,313352.585460,7.591843e+05
0198_Marios_RCC_HP-001300090-1.mzML,8.902157e+05,0.000000,2.188092e+06,0.000000,7.619100e+05,0.000000,1.257957e+06,0.000000,0.000000,6.051173e+05,...,0.000000,219877.785482,640569.903939,465179.273213,992621.938021,591500.332652,0.000000,0.000000e+00,0.000000,4.557918e+05
0001_Marios_PD1_Inhib2_HP-S3473952-7.mzML,0.000000e+00,507466.036163,0.000000e+00,609480.270770,1.129730e+06,288594.347888,1.977715e+06,258031.408399,197663.914116,9.228857e+05,...,405804.599835,154184.219034,0.000000,105841.450804,0.000000,0.000000,513760.986846,6.365423e+05,287373.150214,1.059854e+06


### Select the chosen peaks

recall, that we did some alignment filtering for peak robustness

In [82]:
X_finetune_test = pd.read_csv('/Users/jonaheaton/ReviveMed Dropbox/Jonah Eaton/development_CohortCombination/alignment_RCC_2024_Feb_27/April_05_Data/X_finetune_test.csv',index_col=0)

intensity_data_raw = intensity_data_raw[X_finetune_test.columns].copy()

### Select the Chosen samples

recall, that we did some alignment filtering of samples

In [87]:
metadata_file = 'metadata.csv'
metadata = pd.read_csv(input_dir+'/'+metadata_file,index_col=0)


In [81]:
intensity_data_raw = intensity_data_raw.loc[metadata.index].copy()

In [83]:
intensity_data_raw.shape

(17685, 2736)

In [89]:
adata = ad.AnnData(intensity_data_raw)

In [90]:
adata.var_names

Index(['FT10031', 'FT10035', 'FT10037', 'FT10039', 'FT10041', 'FT10042',
       'FT10044', 'FT1006', 'FT10068', 'FT1007',
       ...
       'FT9951', 'FT9975', 'FT9978', 'FT9980', 'FT9984', 'FT9985', 'FT9988',
       'FT9989', 'FT9997', 'FT9999'],
      dtype='object', length=2736)

In [91]:
adata.obs

""
0582_Marios_PD1_Inhib2_HP-EA003798-7.mzML
0293_Marios_PD1_Inhib2_HP-E9101785-7.mzML
0219_Marios_PD1_Inhib2_HP-EA003799-7.mzML
0962_Marios_PD1_Inhib2_HP-E9784758-7.mzML
0218_Marios_PD1_Inhib2_HP-E9123067-7.mzML
...
1188_Marios_PD1_Inhib2_HP-AE052720-7.mzML
1023_Marios_PD1_Inhib2_HP-AD185798-7.mzML
1093_Marios_PD1_Inhib2_HP-AE726342-7.mzML
0925_Marios_PD1_Inhib2_HP-AE673799-7.mzML


In [93]:
metadata

,cohort_id,Study ID,Cohort Label,Cohort ID,OS,OS_Event,Age,subject ID,study_week,Region,...,is Pediatric,Cohort Label ENC,Study ID ENC,file id,Age Range (min),Age Range (max),IMDC,IMDC ORDINAL,IMDC BINARY,is Female
0582_Marios_PD1_Inhib2_HP-EA003798-7.mzML,541,ST001237,adult_cancer,541,50.562628,0.0,62.0,CA209025-111-657,baseline,REST OF WORLD,...,False,0,4,0582_Marios_PD1_Inhib2_HP-EA003798-7.mzML,62.0,62.0,FAVORABLE,2.0,1.0,1.0
0293_Marios_PD1_Inhib2_HP-E9101785-7.mzML,541,ST001237,adult_cancer,541,24.607803,1.0,59.0,CA209025-33-12,baseline,WESTERN EUROPE,...,False,0,4,0293_Marios_PD1_Inhib2_HP-E9101785-7.mzML,59.0,59.0,INTERMEDIATE,1.0,NaN,0.0
0219_Marios_PD1_Inhib2_HP-EA003799-7.mzML,541,ST001237,adult_cancer,541,52.073922,0.0,66.0,CA209025-111-778,baseline,REST OF WORLD,...,False,0,4,0219_Marios_PD1_Inhib2_HP-EA003799-7.mzML,66.0,66.0,INTERMEDIATE,1.0,NaN,0.0
0962_Marios_PD1_Inhib2_HP-E9784758-7.mzML,541,ST001237,adult_cancer,541,24.246407,1.0,80.0,CA209025-180-1001,baseline,REST OF WORLD,...,False,0,4,0962_Marios_PD1_Inhib2_HP-E9784758-7.mzML,80.0,80.0,POOR,0.0,0.0,1.0
0218_Marios_PD1_Inhib2_HP-E9123067-7.mzML,541,ST001237,adult_cancer,541,3.614000,1.0,60.0,CA209025-113-50,baseline,WESTERN EUROPE,...,False,0,4,0218_Marios_PD1_Inhib2_HP-E9123067-7.mzML,60.0,60.0,NOT REPORTED,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188_Marios_PD1_Inhib2_HP-AE052720-7.mzML,541,ST001237,adult_cancer,541,60.254620,0.0,56.0,CA209025-139-227,baseline,US/CANADA,...,False,0,4,1188_Marios_PD1_Inhib2_HP-AE052720-7.mzML,56.0,56.0,INTERMEDIATE,1.0,NaN,0.0
1023_Marios_PD1_Inhib2_HP-AD185798-7.mzML,541,ST001237,adult_cancer,541,28.977413,1.0,72.0,CA209025-1-27,baseline,US/CANADA,...,False,0,4,1023_Marios_PD1_Inhib2_HP-AD185798-7.mzML,72.0,72.0,FAVORABLE,2.0,1.0,0.0
1093_Marios_PD1_Inhib2_HP-AE726342-7.mzML,541,ST001237,adult_cancer,541,25.856300,1.0,58.0,CA209025-138-872,baseline,US/CANADA,...,False,0,4,1093_Marios_PD1_Inhib2_HP-AE726342-7.mzML,58.0,58.0,FAVORABLE,2.0,1.0,1.0
0925_Marios_PD1_Inhib2_HP-AE673799-7.mzML,541,ST001237,adult_cancer,541,19.679700,1.0,52.0,CA209025-144-717,baseline,US/CANADA,...,False,0,4,0925_Marios_PD1_Inhib2_HP-AE673799-7.mzML,52.0,52.0,FAVORABLE,2.0,1.0,0.0


In [94]:
adata.var["gene_name"] = adata.var.index
adata.obs["Cohort Label"] = metadata.loc[adata.obs.index]['Cohort Label'].astype("category")
adata.obs["Study ID"] = metadata.loc[adata.obs.index]['Study ID'].astype("category")

In [95]:
batch_id_labels = adata.obs["Study ID"].astype("category").cat.codes.values
adata.obs["batch_id"] = batch_id_labels

adata.obs["celltype"] = adata.obs["Cohort Label"]

celltype_id_labels = adata.obs["celltype"].astype("category").cat.codes.values
celltypes = adata.obs["celltype"].unique()
num_types = len(np.unique(celltype_id_labels))
id2type = dict(enumerate(adata.obs["celltype"].astype("category").cat.categories))
adata.obs["celltype_id"] = celltype_id_labels

In [96]:
adata.obs['Cohort Label']

0582_Marios_PD1_Inhib2_HP-EA003798-7.mzML    adult_cancer
0293_Marios_PD1_Inhib2_HP-E9101785-7.mzML    adult_cancer
0219_Marios_PD1_Inhib2_HP-EA003799-7.mzML    adult_cancer
0962_Marios_PD1_Inhib2_HP-E9784758-7.mzML    adult_cancer
0218_Marios_PD1_Inhib2_HP-E9123067-7.mzML    adult_cancer
                                                 ...     
1188_Marios_PD1_Inhib2_HP-AE052720-7.mzML    adult_cancer
1023_Marios_PD1_Inhib2_HP-AD185798-7.mzML    adult_cancer
1093_Marios_PD1_Inhib2_HP-AE726342-7.mzML    adult_cancer
0925_Marios_PD1_Inhib2_HP-AE673799-7.mzML    adult_cancer
0917_Marios_PD1_Inhib2_HP-AD424471-8.mzML    adult_cancer
Name: Cohort Label, Length: 17685, dtype: category
Categories (4, object): ['adult_cancer', 'adult_other', 'pediatric_CMD', 'pediatric_other']

In [97]:
adata.obs.loc[metadata.index, 'Age'] = metadata['Age'].values
adata.obs.loc[metadata.index, 'OS'] = metadata['OS'].values
adata.obs.loc[metadata.index, 'OS_Event'] = metadata['OS_Event'].values
adata.obs.loc[metadata.index, 'PFS'] = metadata['PFS'].values
adata.obs.loc[metadata.index, 'PFS_Event'] = metadata['PFS_Event'].values
adata.obs.loc[metadata.index, 'MSKCC'] = metadata['MSKCC'].astype("category")
adata.obs.loc[metadata.index, 'IMDC'] = metadata['IMDC'].astype("category")

In [98]:
target_data_file = '/Users/jonaheaton/ReviveMed Dropbox/Jonah Eaton/development_CohortCombination/alignment_RCC_2024_Feb_27/rcc_result/matched_targets HILIC POSITIVE ION MODE.csv'

peakinfo_data = pd.read_csv(target_data_file,index_col=0)

In [99]:
peakinfo_subset = peakinfo_data.loc[adata.var.index].copy()
adata.var['mz'] = peakinfo_subset['mzmed']
adata.var['rt (sec)'] = peakinfo_subset['rtmed']
adata.var['auto_corr'] =  peakinfo_subset['auto_corr']
adata.var['cross_corr'] = peakinfo_subset['cross_corr']
adata.var['log2snr'] = peakinfo_subset['log2snr']
adata.var['target_name'] = peakinfo_data['target_name']

In [100]:
namedmatched_file = '/Users/jonaheaton/ReviveMed Dropbox/Jonah Eaton/development_CohortCombination/alignment_RCC_2024_Feb_27/rcc_result/targets_peaks/name_matched_targets.csv'

peakinfo2 = pd.read_csv(namedmatched_file,index_col=0)
common_idx= [x for x in peakinfo2.index if x in adata.var.index]
peakinfo2_subset = peakinfo2.loc[common_idx].copy()

In [101]:
peakinfo2_subset

,mzmed,mzmin,mzmax,rtmed,rtmin,rtmax,auto_corr,cross_corr,freq,chunk_idx,...,target_matched,match_type,mode,input,cleaned,n_match,HMDB_match,MCP_match,fg,metabolite__name
feats,,,,,,,,,,,,,,,,,,,,,
FT129,76.076384,76.057898,76.087775,477.407139,433.527102,549.527268,0.634079,0.940839,0.755152,1.0,...,True,exact synonym,both,trimethylamine-N-oxide,trimethylamine-n-oxide,1,HMDB0000925,MET0000026355,NaN,Trimethylamine N-oxide
FT656,106.050200,106.046581,106.053620,457.768971,441.354660,513.083250,0.915564,0.975459,1.000000,7.0,...,True,exact synonym,both,serine,serine,1,HMDB0000187,MET0000052400,NaN,L-Serine
FT708,110.027359,110.026351,110.028332,475.665336,463.774368,495.832542,0.679665,0.898337,0.961212,8.0,...,True,exact name,both,hypotaurine,hypotaurine,1,HMDB0000965,MET0000033466,NaN,Hypotaurine
FT711,112.050671,112.048886,112.052330,384.677277,371.774358,403.753584,0.641000,0.872566,0.808485,8.0,...,True,exact name,both,cytosine,cytosine,1,HMDB0000630,MET0000047890,NaN,Cytosine
FT943,120.065690,120.058538,120.073110,454.534077,431.929788,531.707436,0.953927,0.991066,1.000000,10.0,...,True,exact synonym,both,threonine,threonine,1,HMDB0000167,MET0000048484,NaN,L-Threonine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FT11568,776.558205,776.544233,776.566249,328.905723,318.549306,375.076338,0.981948,0.993666,1.000000,141.0,...,True,NaN,both,C40:7 PE plasmalogen,c40:7peplasmalogen,0,NaN,NaN,NaN,no_match
FT11669,786.598554,786.589317,786.611756,397.435083,365.182392,550.363410,0.989863,0.912833,1.000000,143.0,...,True,NaN,both,C36:2 PC,c36:2pc,0,NaN,NaN,NaN,no_match
FT11704,792.589955,792.579743,792.596905,390.741276,380.960634,436.785132,0.983792,0.984506,1.000000,144.0,...,True,NaN,both,C38:6 PC plasmalogen,c38:6pcplasmalogen,0,NaN,NaN,NaN,no_match


In [102]:
adata.var.loc[peakinfo2_subset.index, 'HMDB_match'] = peakinfo2_subset['HMDB_match'].values
adata.var.loc[peakinfo2_subset.index, 'MCP_match'] = peakinfo2_subset['MCP_match'].values

In [103]:
adata

AnnData object with n_obs × n_vars = 17685 × 2736
    obs: 'Cohort Label', 'Study ID', 'batch_id', 'celltype', 'celltype_id', 'Age', 'OS', 'OS_Event', 'PFS', 'PFS_Event', 'MSKCC', 'IMDC'
    var: 'gene_name', 'mz', 'rt (sec)', 'auto_corr', 'cross_corr', 'log2snr', 'target_name', 'HMDB_match', 'MCP_match'

In [104]:
adata.obs.loc[metadata.index, 'MSKCC_ordinal_id'] = metadata['MSKCC ORDINAL'].values
adata.obs.loc[metadata.index, 'MSKCC_binary_id'] = metadata['MSKCC BINARY'].values
adata.obs.loc[metadata.index, 'IMDC_ordinal_id'] = metadata['IMDC ORDINAL'].values
adata.obs.loc[metadata.index, 'IMDC_binary_id'] = metadata['IMDC BINARY'].values
adata.obs.loc[metadata.index,'finetune_set'] = metadata['Set'].astype("category")
adata.obs.loc[metadata.index,'pretrain_set'] = metadata['Pretrain'].astype("category")
adata.obs.loc[metadata.index, 'sex'] = metadata['Sex'].astype("category")
adata.obs.loc[metadata.index, 'sex_id'] = metadata['is Female'].values


In [105]:
adata.obs

,Cohort Label,Study ID,batch_id,celltype,celltype_id,Age,OS,OS_Event,PFS,PFS_Event,MSKCC,IMDC,MSKCC_ordinal_id,MSKCC_binary_id,IMDC_ordinal_id,IMDC_binary_id,finetune_set,pretrain_set,sex,sex_id
0582_Marios_PD1_Inhib2_HP-EA003798-7.mzML,adult_cancer,ST001237,4,adult_cancer,0,62.0,50.562628,0.0,18.299795,1.0,FAVORABLE,FAVORABLE,2.0,1.0,2.0,1.0,Test,NaN,F,1.0
0293_Marios_PD1_Inhib2_HP-E9101785-7.mzML,adult_cancer,ST001237,4,adult_cancer,0,59.0,24.607803,1.0,1.905544,1.0,INTERMEDIATE,INTERMEDIATE,1.0,NaN,1.0,NaN,Test,NaN,M,0.0
0219_Marios_PD1_Inhib2_HP-EA003799-7.mzML,adult_cancer,ST001237,4,adult_cancer,0,66.0,52.073922,0.0,29.470226,1.0,INTERMEDIATE,INTERMEDIATE,1.0,NaN,1.0,NaN,Test,NaN,M,0.0
0962_Marios_PD1_Inhib2_HP-E9784758-7.mzML,adult_cancer,ST001237,4,adult_cancer,0,80.0,24.246407,1.0,3.712526,1.0,POOR,POOR,0.0,0.0,0.0,0.0,Test,NaN,F,1.0
0218_Marios_PD1_Inhib2_HP-E9123067-7.mzML,adult_cancer,ST001237,4,adult_cancer,0,60.0,3.614000,1.0,2.759754,1.0,POOR,NOT REPORTED,0.0,0.0,NaN,NaN,Test,NaN,M,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188_Marios_PD1_Inhib2_HP-AE052720-7.mzML,adult_cancer,ST001237,4,adult_cancer,0,56.0,60.254620,0.0,5.683778,1.0,FAVORABLE,INTERMEDIATE,2.0,1.0,1.0,NaN,Finetune,NaN,M,0.0
1023_Marios_PD1_Inhib2_HP-AD185798-7.mzML,adult_cancer,ST001237,4,adult_cancer,0,72.0,28.977413,1.0,5.552361,0.0,FAVORABLE,FAVORABLE,2.0,1.0,2.0,1.0,Finetune,NaN,M,0.0
1093_Marios_PD1_Inhib2_HP-AE726342-7.mzML,adult_cancer,ST001237,4,adult_cancer,0,58.0,25.856300,1.0,9.232033,1.0,FAVORABLE,FAVORABLE,2.0,1.0,2.0,1.0,Finetune,NaN,F,1.0
0925_Marios_PD1_Inhib2_HP-AE673799-7.mzML,adult_cancer,ST001237,4,adult_cancer,0,52.0,19.679700,1.0,5.552361,1.0,FAVORABLE,FAVORABLE,2.0,1.0,2.0,1.0,Finetune,NaN,M,0.0


In [106]:
metadata.head()

,cohort_id,Study ID,Cohort Label,Cohort ID,OS,OS_Event,Age,subject ID,study_week,Region,...,is Pediatric,Cohort Label ENC,Study ID ENC,file id,Age Range (min),Age Range (max),IMDC,IMDC ORDINAL,IMDC BINARY,is Female
0582_Marios_PD1_Inhib2_HP-EA003798-7.mzML,541,ST001237,adult_cancer,541,50.562628,0.0,62.0,CA209025-111-657,baseline,REST OF WORLD,...,False,0,4,0582_Marios_PD1_Inhib2_HP-EA003798-7.mzML,62.0,62.0,FAVORABLE,2.0,1.0,1.0
0293_Marios_PD1_Inhib2_HP-E9101785-7.mzML,541,ST001237,adult_cancer,541,24.607803,1.0,59.0,CA209025-33-12,baseline,WESTERN EUROPE,...,False,0,4,0293_Marios_PD1_Inhib2_HP-E9101785-7.mzML,59.0,59.0,INTERMEDIATE,1.0,NaN,0.0
0219_Marios_PD1_Inhib2_HP-EA003799-7.mzML,541,ST001237,adult_cancer,541,52.073922,0.0,66.0,CA209025-111-778,baseline,REST OF WORLD,...,False,0,4,0219_Marios_PD1_Inhib2_HP-EA003799-7.mzML,66.0,66.0,INTERMEDIATE,1.0,NaN,0.0
0962_Marios_PD1_Inhib2_HP-E9784758-7.mzML,541,ST001237,adult_cancer,541,24.246407,1.0,80.0,CA209025-180-1001,baseline,REST OF WORLD,...,False,0,4,0962_Marios_PD1_Inhib2_HP-E9784758-7.mzML,80.0,80.0,POOR,0.0,0.0,1.0
0218_Marios_PD1_Inhib2_HP-E9123067-7.mzML,541,ST001237,adult_cancer,541,3.614000,1.0,60.0,CA209025-113-50,baseline,WESTERN EUROPE,...,False,0,4,0218_Marios_PD1_Inhib2_HP-E9123067-7.mzML,60.0,60.0,NOT REPORTED,NaN,NaN,0.0


In [107]:
adata.var

,gene_name,mz,rt (sec),auto_corr,cross_corr,log2snr,target_name,HMDB_match,MCP_match
FT10031,FT10031,621.440839,105.834594,0.913195,0.966755,4.816396,NaN,NaN,NaN
FT10035,FT10035,624.944482,320.532930,0.911199,0.974067,4.943304,NaN,NaN,NaN
FT10037,FT10037,620.436915,105.828786,0.970204,0.905678,7.533200,NaN,NaN,NaN
FT10039,FT10039,624.442226,320.542416,0.945020,0.985359,5.460968,NaN,NaN,NaN
FT10041,FT10041,621.303424,348.308292,0.908559,0.978142,4.434385,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
FT9985,FT9985,615.245635,137.580237,0.925210,0.861749,4.656519,NaN,NaN,NaN
FT9988,FT9988,615.539979,101.891388,0.817712,0.921555,3.724821,NaN,NaN,NaN
FT9989,FT9989,616.946874,292.077120,0.865696,0.961949,5.022671,NaN,NaN,NaN
FT9997,FT9997,617.936620,331.764546,0.822638,0.949867,4.196100,NaN,NaN,NaN


In [110]:
adata.X

array([[      0.        ,       0.        ,       0.        , ...,
              0.        ,       0.        ,  389871.65742334],
       [      0.        ,       0.        ,       0.        , ...,
              0.        ,       0.        ,  612885.66422424],
       [      0.        ,       0.        ,       0.        , ...,
              0.        ,       0.        ,  854382.54503697],
       ...,
       [1673977.09843351,  523857.977952  , 4488959.71840198, ...,
         454614.63691946,  298447.48870316, 1263003.09939194],
       [ 884613.46160191,  576978.07908902, 2324544.1998464 , ...,
         751143.55222863,  435310.64038906,  625805.972566  ],
       [1773311.56775302,  542087.94571332, 4800560.61805985, ...,
         822651.09381306,  436126.27293064,  736797.9740457 ]])

In [114]:
# adata.write(f'{input_dir}/data.h5ad', compression="gzip")

In [118]:
from scipy.sparse import csr_matrix

# Convert the DataFrame to a CSR matrix
csr_matrix_data = csr_matrix(adata.X)

# Print the converted CSR matrix
adata.X = csr_matrix_data

In [119]:
adata.write(f'{input_dir}/data.h5ad', compression="gzip")